In [46]:
import torch.nn as nn
import numpy as np
import torch.utils.data as Data
import torch
import torch.optim as optim

In [47]:
# 手写数字训练数据集
mnist = np.load("../../../../../Other/datasets/NumPy/mnist.npz")
X_train, y_train, X_val, y_val = torch.unsqueeze(torch.tensor(mnist['x_train']), 1).to(
    dtype=torch.float32), torch.tensor(mnist['y_train']).to(dtype=torch.long), torch.unsqueeze(
    torch.tensor(mnist['x_test']), 1).to(dtype=torch.float32), torch.tensor(mnist['y_test']).to(dtype=torch.long)
X_train, X_val = X_train / 255.0, X_val / 255.0

In [48]:
batch_size = 200
train_data = Data.TensorDataset(X_train, y_train)
train_loader = Data.DataLoader(dataset=train_data,
                               batch_size=batch_size,
                               shuffle=True)
val_data = Data.TensorDataset(X_val, y_val)
val_loader = Data.DataLoader(dataset=val_data, batch_size=batch_size)

In [49]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.featrues = nn.Sequential(  # 内部实现了forward函数;各模块顺序执行
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),  # 10分类问题
        )

    def forward(self, x):
        x = self.featrues(x)
        x = x.reshape(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [50]:
# 所有的模型,数据必须都切换到同一设备上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criteon = nn.CrossEntropyLoss()
net = LeNet()

# 转换module的devic和dtype(in-place)
net.to(device=device, dtype=torch.float32)  # 模型设备切换必须在优化器定义前执行
optimizer = optim.SGD(net.parameters(), lr=0.001)

for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    print(data.device, target.device)
    break

cuda:0 cuda:0
